# Module 3 Assignment

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `# YOUR CODE HERE`. Do not write your answer anywhere else other than where it says `# YOUR CODE HERE`. Anything you write elsewhere will be removed or overwritten by the autograder.
2. Before you submit your assignment, make sure everything runs as expected. Go to the menubar, select Kernel, and restart the kernel and run all cells (Restart & Run all).
3. Do not change the title (i.e. file name) of this notebook.
4. Make sure that you save your work (in the menubar, select File → Save and CheckPoint).
5. All work must be your own, if you do use any code from another source (such as a course notebook or a website) you need to properly cite the source.

-----

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from nose.tools import assert_equal, assert_almost_equal, assert_true, assert_is_instance

-----

## Loading Breast Cancer Data

In this assignment, we will work with a breast cancer data set to make predictive models. Before we build a model, we first load the data into the assignment notebook, and randomly sample several rows. The second Code cell splits the DataFrame into a training and testing data set, respectively, before creating the features and labels to use for our classification task.

-----

In [ ]:
# Load data
df = pd.read_csv('./breast-cancer-wisconsin.data')
df.sample(5)

In [ ]:
# Split data into training and testing DataFrames
train, test = train_test_split(df, test_size=0.25, random_state=0)

# Create training features and label
y = train['class']
X = train.drop('class', axis=1)

# Create testing features and label
yTest = test['class']
XTest = test.drop('class', axis=1)

-----

## Problem 1: Creating a Random Forest Classifier

For this problem, you will complete the `classify` function, provided below, to complete the following tasks:
- Create a random forest classifier by using the `RandomForestClassifier` estimator in the scikit learn library.
- When creating the RandomForestClassifier estimator assign the `s_estimators` hyperparameter to the `ne` parameter, assign the `random_state` hyperparameter to the `rs` parameter, and leave the other parameters as their default values.
- Fit this `RandomForestClassifier` estimator to the X (which are the features) and y (which are the labels) DataFrames.
- Return the RandomForestClassifier model.

-----

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def classify(X, y, rs=0, ne=10):
    '''
    Create and fit a RandomForestClassifier estimator to training data.
    
    Parameters
    ---------
    X : Pandas DataFrame
    y : Pandas DataFrame
    rs: seed for random number generator
    ne: float for number of estimators
    
    Returns
    -------
    The RandomForestClassifier estimator
    '''
    
    ### YOUR CODE HERE

In [ ]:
# Classify data by using different numbers of tress in the forest
rfc1 = classify(X, y, rs=0, ne=1)
rfc5 = classify(X, y, rs=0, ne=5)
rfc10 = classify(X, y, rs=0, ne=10)
rfc20 = classify(X, y, rs=0, ne=20)

# Check solutions
assert_almost_equal(rfc1.score(XTest,yTest), 0.9356, places=2)
assert_almost_equal(rfc5.score(XTest,yTest), 0.9532, places=2)
assert_almost_equal(rfc10.score(XTest,yTest), 0.9415, places=2)
assert_almost_equal(rfc20.score(XTest,yTest), 0.9473, places=2)

-----

We now load a raw version of the breast cancer data set, which we will use in the remaining two problems. The first Code cell below loads these data and samples several instances. The second Code cell generates features and  labels.

-----

In [ ]:
# Load uncleaned data
df = pd.read_csv('./breast-cancer-wisconsin-not-cleaned.data')
df.sample(5)

In [ ]:
# Create labels (y) and features (X)
y = df['class']
X = df.drop('class', axis=1)

-----

## Problem 2: Creating a Data Pre-Processing Pipeline

Previously, we have often cleaned data, such as the breast cancer dataset, by removing rows that contained a _NaN_. In this problem, however, we will instead create a pipeline that replaces, or imputes, missing values (i.e., _NaN_) with the mean value of the appropriate column. After this, the pipeline will apply a standard scaling to the columns. Thus, to complete this problem, you must explicitly:
- Create an `Imputer` by using the scikit learn library. When creating this object, set the `missing_values` argument to `'NaN'`, the `strategy` argument to `'mean'`, the `axis` argument to `0`, and set `copy=False`.
- Create a `StandardScaler` by using the scikit learn library, leaving the parameters as their default values.
- Create a pipeline. **Important:** The first item in the pipeline must be the `Imputer` with the name `'imp'`, while the second item in the pipeline must be the `StandardScaler` use the name `'ss'`.
- Apply the pipeline to fit and transform the features (X) and the labels (y).
- Return the preprocessing pipeline and the transformed  features.

-----

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler

def preprocess(X, y):
    '''
    Create and fit a RandomForestClassifier pipeline to training data, 
    and compute the mean accuracy score from the testing data.
    
    Parameters
    ---------
    X : Pandas DataFrame
    y : Pandas DataFrame

    Returns
    -------
    The pipeline used to clean the data and the cleaned features as a NumPy array
    '''

    ### YOUR CODE HERE

In [ ]:
# Apply pipeline to uncleaned data
data_pipeline, new_X = preprocess(X, y)

# Test pipeline
assert_equal(type(data_pipeline), Pipeline, 
             msg='Please return a pipeline object as the first object')

assert_equal(new_X[30][0], -0.0012472123989550703)
assert_equal(new_X[100][0], 0.15397593605209656)
assert_equal(new_X[123][0], 0.16658267824600242)
assert_equal(new_X[256][0], 0.17951699465179008)
assert_equal(new_X[512][0], 0.37020734898435897)

assert_equal(type(data_pipeline.get_params()['imp']), type(Imputer()), 
             msg='You  did not label the imputer "imp" or did not make an Imputer Object')

assert_equal(type(data_pipeline.get_params()['steps'][0][1]), Imputer, 
             msg='You need to create a Imputer First')

assert_equal(data_pipeline.get_params()['imp__missing_values'], 'NaN', 
             msg='set missing values to NaN')

assert_equal(data_pipeline.get_params()['imp__strategy'], 'mean', 
             msg='Set strategy to mean')

assert_equal(type(data_pipeline.get_params()['ss']), type(StandardScaler()), 
             msg='You  did not create a Standard Scaler labeled "ss" ')

-----

We now need to split the newly cleaned data into training and testing data sets in order to create and apply a classifier pipeline to these data.

-----

In [ ]:
trainX, testX, trainY, testY = train_test_split(new_X, y, random_state=0)

-----

## Problem 3: Creating a Random Forest Pipeline

For this problem, you will finish the `rfcp` template function provided below by creating a random forest pipeline. This will require that you complete the following tasks:
- Create a random forest classifier by using the scikit learn RandomForestClassifer estimator. Do not change any of the default hyperparameters for this estimator.
- Create a pipeline that includes this random forest classifier, and label the classifier `rfc`.
- Set the random_state of the random forest classifier in the pipeline to the value specified by the `rs` parameter.
- Fit the pipeline to the training data.
- Calculate the mean accuracy of the random forest classifier in the pipeline from the test data.
- Return the pipeline and the mean accuracy score.

-----

In [ ]:
def rfcp(trainX, trainY, testX, testY, rs=0):
    '''
    Create and fit a RandomForestClassifier pipeline to training data, 
    and compute the mean accuracy score from the testing data.
    
    Parameters
    ---------
    trainX : NumPy array (features)
    trainY : NumPy array (labels)
    testX : NumPy array (features)
    testY : NumPy array (labels)
    rs: random state

    Returns
    -------
    The pipeline and a float containing the mean accuracy score
    '''

    ### YOUR CODE HERE

In [ ]:
# Create and apply the RFC pipeline
ml_pipeline, score = rfcp(trainX, trainY, testX, testY, rs=0)

# Test the pipeline
assert_equal(type(ml_pipeline.get_params()['rfc']), type(RandomForestClassifier()))
assert_almost_equal(score, 0.9555, places=2)

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode 